In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib.font_manager as fm
from shapely.geometry import Polygon, Point
from geopy.distance import great_circle
from geopy.point import Point
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
#from geopandas import GeoDataFrame
#from descartes import PolygonPatch
import geopy
import datetime
import os
import warnings
import notebook
import pickle
import seaborn as sns
warnings.filterwarnings('ignore')
E = notebook.nbextensions.EnableNBExtensionApp()
E.enable_nbextension('usability/codefolding/main')
E.enable_nbextension('usability/hide_input/main')
E.enable_nbextension('usability/hide_input_all/main')
E.enable_nbextension('usability/highlighter/highlighter')
E.enable_nbextension('usability/toggle_all_line_numbers/main')
E.enable_nbextension('usability/rubberband/main')
E.enable_nbextension('usability/execute_time/ExecuteTime')
from lightning import Lightning
#lgn = Lightning(ipython=True, local=True)
import codecs, json ,base64
import simplejson
import IPython.core.display as di
from ggplot import *
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
from shapely.prepared import prep
import fiona
from matplotlib.collections import PatchCollection
from descartes import PolygonPatch
import json
import datetime

/home/ecoker/.local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
##  This reads in the Crash from 2012 thru 2014 data, filters it to Chicago and only looks at those which are Pedestrian or Pedacyclist

frame=pd.read_csv('CrashExtract2014.txt',error_bad_lines=False)
frame2=pd.read_csv('CrashExtract2013.txt',error_bad_lines=False)
frame3=pd.read_csv('CrashExtract2012.txt',error_bad_lines=False)
frame=frame.iloc[:,(1,10,12,13,14,26,28,29,33,35,36,37,40,41,46,47,48,49,54,55,60,64,67)]
frame2=frame2.iloc[:,(1,10,12,13,14,26,28,29,33,35,36,37,40,41,46,47,48,49,54,55,60,64,67)]
frame3=frame3.iloc[:,(1,10,12,13,14,26,28,29,33,35,36,37,40,41,46,47,48,49,54,55,60,64,67)]
frame.columns=['id','city','accident_type','total_killed','total_injured','road_surface','lighting','weather','times','intersection','hit_run','dayte','trafficway','roadway','latitude','longitude','county','day_of_wk','cause1','cause2','crash_severity','trafficway_d','crash_severity_cd']
frame2.columns=['id','city','accident_type','total_killed','total_injured','road_surface','lighting','weather','times','intersection','hit_run','dayte','trafficway','roadway','latitude','longitude','county','day_of_wk','cause1','cause2','crash_severity','trafficway_d','crash_severity_cd']
frame3.columns=['id','city','accident_type','total_killed','total_injured','road_surface','lighting','weather','times','intersection','hit_run','dayte','trafficway','roadway','latitude','longitude','county','day_of_wk','cause1','cause2','crash_severity','trafficway_d','crash_severity_cd']
frames=pd.concat([frame,frame2,frame3],axis=0)
frames=frames[frames.city==3]
frames=frames[(frames.accident_type==1) | (frames.accident_type==2)]
frames['longitude']=frames['longitude']*-1
frames['datetime_accident'] = pd.to_datetime(frames['dayte']+ ' ' + frames['times'])
frames['daytetime'] = frames['datetime_accident'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))
colskeep = ['id','daytetime','datetime_accident','accident_type','road_surface','lighting','weather','intersection','hit_run','trafficway','roadway','latitude','longitude',\
           'day_of_wk','cause1','cause2','crash_severity']
frames = frames.loc[:,colskeep]
#frames.info()
pickle.dump(frames, open( "accident_info_2012-13-14.p", "wb" ))

In [ ]:
## City of Chicago Liquor Licenses data use geolocator to add Coordinates

liqu = pd.read_csv('Business_Licenses_-_Current_Liquor_and_Public_Places_of_Amusement_Licenses (1).csv',infer_datetime_format=True,parse_dates=['DATE ISSUED'])
liqu = liqu.loc[:,['DOING BUSINESS AS NAME','ADDRESS','DATE ISSUED','LATITUDE','LONGITUDE']]
liqu.columns = ['biz_name','address','daytetime','latitude','longitude']
liqfix = liqu[liqu.latitude.isnull()]
liqu = liqu.drop_duplicates().dropna()

geolocator = Nominatim()
try:
    liqfix['coords'] = (liqfix['address']).apply(geolocator.geocode,timeout=1000)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(liqfix['coords'], e.msg))
liqcols = ['biz_name','address','daytetime','latitude','longitude']
liqu = liqu.loc[:,liqcols]
liqfix = liqfix.loc[:,liqcols]
liq = pd.concat([liqu,liqfix],axis=0)
liq = liq.dropna(subset=['latitude','longitude'])
liq = liq[liq.latitude.notnull()]
liq.head()

In [3]:
## Traffic Segments by Coordinate (polygon)
segments = pd.read_csv('Segments.csv')
segments.head()
segments.columns=['segment_id','STREET','direction','FROM_STREET','TO_STREET','length','STREET_HEADING','COMMENTS','START_LONGITUDE','START_LATITUDE','END_LONGITUDE','END_LATITUDE','CURRENT_SPEED','datetime']
segments['lon']=((segments['START_LONGITUDE']+segments['END_LONGITUDE']) / 2)
segments['lat']=((segments['START_LATITUDE']+segments['END_LATITUDE'] / 2))
keeping = ['segment_id','direction','lat','lon','length','START_LATITUDE','END_LATITUDE','START_LONGITUDE','END_LONGITUDE']
segments = segments.loc[:,keeping]

In [4]:
## Historical Congestion from City of Chicago Data
ctcs = pd.read_csv('Historical_Congestion_by_Seg.csv',infer_datetime_format=True,parse_dates=['daytetime'])
dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M:%S %p')
ctcs2 = pd.read_csv('Chicago_Traffic_Tracker_-_Historical_Congestion_Estimates_by_Segment (1).csv',infer_datetime_format=True,parse_dates=['TIME'],date_parser=dateparse,engine='c')
ctcs2.columns = ['daytetime', 'segment_id', 'bus_ct', 'message_ct', 'speed', 'id']
tokeep=['daytetime','segment_id','speed']
ctcs2=ctcs2.loc[:,tokeep]
#ctcs.to_csv('Historical_Congestion_by_Seg.csv',index=False)
cong = pd.DataFrame()
cong = pd.concat([ctcs,ctcs2],axis=0)
traffic = pd.merge(cong,segments,how='inner',on=['segment_id'])
pickle.dump(traffic, open( "traffic.p", "wb" ))

In [6]:
%load_ext Cython

In [ ]:
## Use Cython to create datetime part calculations. Cython is used frequently along with pickle to make some dataframe operations quicker.

In [ ]:
%%cython -a
import pandas as pd
import pickle
import datetime
traffic = pickle.load( open( "traffic.p", "rb" ) )
traffic['daytetime'] = pd.to_datetime(traffic['daytetime'])
traffic['dayte'] = traffic['daytetime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day))
traffic['hour'] = traffic['daytetime'].apply(lambda x: x.hour)
traffic['dow'] = traffic['daytetime'].apply(lambda x: x.weekday())
traffic['avg_day_spd'] = traffic.groupby(['segment_id','dow','direction'])['speed'].transform(lambda x: x.mean())
traffic['avg_dly_spd'] = traffic.groupby(['segment_id','dayte','direction'])['speed'].transform(lambda x: x.mean())
traffic['avg_h_spd'] = traffic.groupby(['segment_id','hour','direction'])['speed'].transform(lambda x: x.mean())
traffic['datetime_recorded'] = traffic['daytetime']
traffic['daytetime'] = traffic['datetime_recorded'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))
pickle.dump(traffic, open( "traffic2.p", "wb" ))

In [ ]:
## This will start to merge traffic with accident data. Then, bring in liquor data and use Cython/pickling to calculate distances between accidents and liquor establishments nearby.
traffic = pickle.load( open( "traffic2.p", "rb" ) )
merge1 = pd.merge(traffic,frames,how='right',on='daytetime')
merge1 = merge1.drop_duplicates(subset=['id','avg_day_spd','avg_dly_spd','avg_h_spd'])
mergekeep = ['latitude','longitude','id']
merge1.loc[:,mergekeep].head()
geocolsframe = pd.DataFrame()
geocolsframe2 = pd.DataFrame()
geoframe = pd.DataFrame()
mergekeep = ['latitude','longitude','id']
geocolsframe = merge1.loc[:,mergekeep]
geocolsframe = geocolsframe.drop_duplicates()
liqcols = ['biz_name','latitude','longitude']
geocolsframe2 = liq.loc[:,liqcols]
geoframe = pd.concat([geocolsframe,geocolsframe2],axis=0)
#geoframe['coords']=zip(geoframe.latitude,geoframe.longitude)
#del geoframe['latitude'],geoframe['longitude']
del geocolsframe,geocolsframe2

geoframe['id'] = geoframe['id'].apply(lambda x: 'id' + ' ' + str(x))
geoframe['biz_name'] = geoframe['biz_name'].apply(lambda x: 'biz_name' + ' ' + str(x))
geoframe = pd.melt(geoframe,id_vars=['latitude','longitude'])
print geoframe.head()

geoframe = geoframe[~geoframe['value'].str.contains('nan')]
geoframe = geoframe.dropna()
del geoframe['variable']

geoframe = geoframe.drop_duplicates()
pickle.dump(geoframe, open( "coords.p", "wb" ))

geoframe_single = geoframe.value.drop_duplicates()
pickle.dump(geoframe_single, open( "geoframe.p", "wb" ))

In [ ]:
%reload_ext Cython

In [ ]:
%%cython -a
import numpy as np
import pandas as pd
import pickle

def expand_grid(x, y):
    xG, yG = np.meshgrid(x, y) # create the actual grid
    xG = xG.flatten() # make the grid 1d
    yG = yG.flatten() # same
    return pd.DataFrame({'x':xG, 'y':yG})

geoframe_single = pickle.load( open( "geoframe.p", "rb" ) )
liquor = geoframe_single[geoframe_single.str.startswith('biz_name')]
accident = geoframe_single[geoframe_single.str.startswith('id')]
liquor = liquor.dropna()
accident = accident.dropna()
liq = liquor.as_matrix()
acc = accident.as_matrix()

distances = expand_grid(liq, acc)
pickle.dump(distances, open("save.p", "wb"))
print len(distances),distances.head()

In [ ]:
%reload_ext Cython

In [ ]:
%%cython -a
import numpy as np
import geopy
from geopy.distance import great_circle
from itertools import product
import pickle
import itertools
import pandas as pd

coords = pickle.load( open( "coords.p", "rb" ) )
distances = pickle.load(open("save.p", "rb"))
distances.columns = ['biz_name','id']
liquor = coords[coords.value.str.startswith('biz_name')]
accident = coords[coords.value.str.startswith('id')]
distance_frame_pre = pd.DataFrame()
distance_frame_pre2 = pd.DataFrame()
distance_frame_pre = pd.merge(liquor,distances,how='right',left_on='value',right_on='biz_name')
distance_frame_pre2 = pd.merge(accident,distances,how='right',left_on='value',right_on='id')
distance_frame_pre.columns = ['biz_lat','biz_lon','value','biz_name','id']
del distance_frame_pre['value']
distance_frame_pre2.columns = ['id_lat','id_lon','value','biz_name','id']
del distance_frame_pre2['value']
distances_frame = pd.merge(distance_frame_pre,distance_frame_pre2,how='inner',on=['biz_name','id']).dropna()
pickle.dump(distances_frame, open( "distances.p", "wb" ) )

In [ ]:
%%cython -a
import numpy as np
from geopy.distance import great_circle
import geopy
import pandas as pd
import random
import pickle

distances_frame = pickle.load(open("distances.p", "rb"))
distances_frame['distances'] = distances_frame.apply(lambda x: great_circle((x['biz_lat'],x['biz_lon']), (x['id_lat'], x['id_lon'])).miles, axis=1)
thecols = ['biz_name','id','distances']
distances_frame = distances_frame.loc[:,thecols].drop_duplicates()
distance_data = pd.DataFrame()
distance_data = distances_frame[distances_frame['distances'] < 1.1 ]
distance_data['ct_1m'] = distance_data.groupby('id')['biz_name'].transform(lambda x: x.nunique())
pickle.dump(distance_data,open("distance_data.p","wb"))
pickle.dump(distances_frame, open( "distances_full.p", "wb" ))